# We divide the data set into 2 to 10 clusters by kmeans

## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cp '/content/gdrive/MyDrive/Colab Notebooks/ml_project_dataset.pkl' '/content/'

In [ ]:
!pip install pickle5

     |████████████████████████████████| 133kB 27.3MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219267 sha256=e9afbfba3a4b825f278f25cac13d965f32c2f7020d442c87edab662fb242eabd
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
!pip install colorama

In [ ]:
import pickle5 as pickle
import numpy as np
import pandas as pd

with open('ml_project_dataset.pkl', "rb") as fh:
  data = pickle.load(fh)

## Show Dataset

In [ ]:
data.head()

,Age,Gender,Status,f1,f2,f3,f4,f5
0,28,Female,Happy,"[-0.03723609, -0.020824267, -0.025246128, 0.00...","[-0.0013615170028060675, -0.000550377124454826...","[-0.045345112681388855, 0.049214184284210205, ...","[0.622996985912323, 0.5587776899337769, 0.2035...","[142.0, 268.0, -91.8211441040039, 147.0, 304.0..."
1,24,Male,Sad,"[-0.009315872, -0.0008500488, 0.0008058883, 0....","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.22100389003753662, 0.0, 0.0, 0.070420034229...","[1302.0, 2091.0, -218.44000244140625, 928.0, 2..."
2,29,M,neutral,"[-0.013662366, 0.03466877, -0.065040134, -0.04...","[-0.008576362393796444, 0.09807106107473373, -...","[-0.002319525694474578, 0.03656185418367386, -...","[0.0, 0.21522775292396545, 0.26262786984443665...","[831.0, 252.0, -65.05692291259766, 909.0, 516...."
3,28,M,happy,"[-0.06278418, -0.03465886, -0.035757925, -0.02...","[0.03950166329741478, 0.1211700439453125, 0.03...","[-0.02030731551349163, 0.0438639298081398, -0....","[0.021185416728258133, 0.015900805592536926, 0...","[796.0, 715.0, -420.5904541015625, 850.0, 845...."
4,25,Male,Normal,"[0.027600960806012154, 0.03169381991028786, -0...","[-0.012523038312792778, 0.1949128955602646, 0....","[-0.024486687034368515, 0.031889624893665314, ...","[0.36306896805763245, 0.45892614126205444, 0.1...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


## Preprocess for categorical data

In [ ]:
import re
data.Gender.unique()
data.Gender=data.Gender.apply(lambda x: re.sub('^[fF]\w*','f',x))
data.Gender=data.Gender.apply(lambda x: re.sub('^[mM]\w*','m',x))

data.Status = data.Status.apply(lambda x : re.sub('^[sS]+.*','s',x))
data.Status = data.Status.apply(lambda x : re.sub('^[hH]+.*','h',x))
data.Status = data.Status.apply(lambda x : re.sub('^[nN]+.*','n',x))

#find cropted label and drop the row data thats containe it 
data.Status = data.Status.apply(lambda x : re.sub('^\d+.*','cropted',x))
data.Status.replace('cropted',np.nan,inplace=True)
data.dropna(subset=['Status'],inplace=True)

data.Gender=data.Gender.astype('category').cat.codes
data.Status=data.Status.astype('category').cat.codes

## Clean NaN data

In [ ]:
data_f1 = data[['Age','Gender','Status','f1']].copy()
data_f2 = data[['Age','Gender','Status','f2']].copy()
data_f3 = data[['Age','Gender','Status','f3']].copy()
data_f4 = data[['Age','Gender','Status','f4']].copy()
data_f5 = data[['Age','Gender','Status','f5']].copy()

In [ ]:
data_f1.f1=data_f1.f1.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f1.dropna(subset=['f1'],inplace=True)

data_f2.f2=data_f2.f2.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f2.dropna(subset=['f2'],inplace=True)

data_f3.f3=data_f3.f3.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f3.dropna(subset=['f3'],inplace=True)

data_f4.f4=data_f4.f4.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f4.dropna(subset=['f4'],inplace=True)

data_f5.f5=data_f5.f5.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f5.dropna(subset=['f5'],inplace=True)

data_f1.reset_index(drop=True,inplace=True)
data_f2.reset_index(drop=True,inplace=True)
data_f3.reset_index(drop=True,inplace=True)
data_f4.reset_index(drop=True,inplace=True)
data_f5.reset_index(drop=True,inplace=True)

## F1

In [ ]:
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from collections import Counter
import pandas as pd
from sklearn.cluster import KMeans

def make_cluster_with_PCA(X,feature_set,nc):
  x=X[feature_set].values.copy()
  u = np.zeros((X.shape[0],x[0].shape[0]))
  for i in range(X.shape[0]):
    for j in range(x[0].shape[0]):
      u[i][j]=x[i][j]
  
  cols = np.arange(0,x[0].shape[0],1)
  df = pd.DataFrame(u, columns = cols)
  pca = PCA()
  x_pca=pca.fit_transform(df)

  exv=pca.explained_variance_ratio_
  yy = []
  a,s=0,0
  flag =True 
  for k,j in enumerate(exv):
    s+=j
    if (s>0.95 and flag):
      a = k
      flag=False
    yy.append(s)

  pca = PCA(a)
  x_pca=pca.fit_transform(df)
  
  SS = StandardScaler()
  for i in range(a):
    x_pca[:,i]=SS.fit_transform(x_pca[:,i].reshape(-1,1)).reshape(-1,)

  kmean = KMeans(n_clusters=nc, random_state=0)
  kmean.fit(x_pca)
  pred = kmean.predict(x_pca)
  return x_pca , pred

def make_cluster_without_PCA(X,feature_set,nc):
  x=X[feature_set].values.copy()
  u = np.zeros((X.shape[0],x[0].shape[0]))
  for i in range(X.shape[0]):
    for j in range(x[0].shape[0]):
      u[i][j]=x[i][j]
  
  cols = np.arange(0,x[0].shape[0],1)
  df = pd.DataFrame(u, columns = cols)
  x = df.values
  SS = StandardScaler()
  for i in range(x.shape[1]):
    x[:,i]=SS.fit_transform(x[:,i].reshape(-1,1)).reshape(-1,)

  kmean = KMeans(n_clusters=nc, random_state=0)
  kmean.fit(x)
  pred = kmean.predict(x)
  return x , pred


In [ ]:
import colorama
from colorama import Fore,Style

def clustering_information(data,feature_set,use_pca):
  print(Fore.CYAN+Style.BRIGHT+'\u007B'+f' Clustring in Feature Set {feature_set}'+' \u007D'+Style.RESET_ALL)
  target_name = {3: Fore.GREEN+'\n>>'+' Gender Category :'+Style.RESET_ALL,2: Fore.GREEN+'\n\n>>'+' Status Category :'+Style.RESET_ALL,
                 1: Fore.GREEN+'\n\n>>'+' Age Category :'+Style.RESET_ALL}
  
  for nc in range(2 , 11):
    print(Fore.CYAN+'\n>'+f' number of clusters = {nc}'+Style.RESET_ALL)
    X = data.drop(columns=['Age','Gender','Status'])
    if use_pca:
      XX,YY=make_cluster_with_PCA(X,feature_set,nc)
    else:
      XX,YY=make_cluster_without_PCA(X,feature_set,nc)
    x_ = np.concatenate((XX,data.Gender.values.reshape(-1,1),data.Status.values.reshape(-1,1),
                          data.Age.values.reshape(-1,1),YY.reshape(-1,1)),axis=1)
    
    for k in range(3 ,0, -1):
      print(target_name[k])
      clus ={}
      for i in range(nc):
        clus[i]=x_[np.where(x_[:,-1]==i)][:,:-k]
        f=dict(Counter(clus[i][:,-1]))
        sort_orders = sorted(f.items(), key=lambda x: x[1], reverse=True)

        print(f'\n>>> in cluster {i}',end='')
        for item in sort_orders:
          print(' ',item,end='')
    print('\n\n',Fore.YELLOW+Style.BRIGHT+30*'--'+Style.RESET_ALL)

In [ ]:
clustering_information(data_f1 , 'f1',use_pca=True)

{ Clustring in Feature Set f1 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 826)  (0.0, 173)
>>> in cluster 1  (0.0, 647)  (1.0, 596)

>> Status Category :

>>> in cluster 0  (1.0, 341)  (2.0, 335)  (0.0, 323)
>>> in cluster 1  (0.0, 418)  (1.0, 414)  (2.0, 411)

>> Age Category :

>>> in cluster 0  (25.0, 122)  (23.0, 111)  (24.0, 96)  (26.0, 72)  (52.0, 51)  (28.0, 36)  (22.0, 35)  (58.0, 34)  (56.0, 32)  (60.0, 30)  (38.0, 30)  (32.0, 29)  (29.0, 22)  (46.0, 22)  (31.0, 21)  (6.0, 20)  (51.0, 19)  (21.0, 15)  (30.0, 15)  (59.0, 14)  (17.0, 14)  (16.0, 13)  (40.0, 13)  (55.0, 12)  (20.0, 12)  (34.0, 11)  (10.0, 11)  (49.0, 11)  (27.0, 11)  (18.0, 10)  (5.0, 10)  (14.0, 9)  (7.0, 8)  (44.0, 5)  (57.0, 5)  (47.0, 4)  (11.0, 4)  (62.0, 3)  (8.0, 3)  (9.0, 2)  (12.0, 2)
>>> in cluster 1  (23.0, 156)  (22.0, 96)  (25.0, 88)  (24.0, 82)  (28.0, 60)  (21.0, 52)  (34.0, 46)  (54.0, 45)  (50.0, 45)  (52.0, 38)  (26.0, 34)  (20.0, 33)  (11.0, 32)  (35.0, 30)  (30.0

In [ ]:
clustering_information(data_f1 , 'f1',use_pca=False)

{ Clustring in Feature Set f1 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 1140)  (0.0, 17)
>>> in cluster 1  (0.0, 803)  (1.0, 282)

>> Status Category :

>>> in cluster 0  (1.0, 397)  (2.0, 380)  (0.0, 380)
>>> in cluster 1  (2.0, 366)  (0.0, 361)  (1.0, 358)

>> Age Category :

>>> in cluster 0  (23.0, 204)  (25.0, 128)  (24.0, 107)  (22.0, 75)  (26.0, 67)  (52.0, 60)  (28.0, 51)  (29.0, 48)  (58.0, 36)  (56.0, 30)  (32.0, 30)  (38.0, 30)  (30.0, 30)  (46.0, 27)  (59.0, 27)  (16.0, 25)  (51.0, 18)  (57.0, 16)  (34.0, 15)  (60.0, 15)  (20.0, 15)  (18.0, 15)  (49.0, 15)  (31.0, 15)  (21.0, 12)  (17.0, 11)  (40.0, 11)  (55.0, 6)  (5.0, 4)  (54.0, 4)  (8.0, 4)  (11.0, 3)  (47.0, 1)  (6.0, 1)  (14.0, 1)
>>> in cluster 1  (25.0, 82)  (24.0, 71)  (23.0, 63)  (22.0, 56)  (21.0, 55)  (6.0, 48)  (28.0, 45)  (50.0, 45)  (34.0, 42)  (54.0, 41)  (26.0, 39)  (44.0, 34)  (11.0, 33)  (20.0, 30)  (35.0, 30)  (40.0, 29)  (52.0, 29)  (55.0, 24)  (14.0, 22)  (7.0, 21)  (16

In [ ]:
clustering_information(data_f2 , 'f2',use_pca=True)

{ Clustring in Feature Set f2 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 1375)  (0.0, 167)
>>> in cluster 1  (0.0, 671)  (1.0, 70)

>> Status Category :

>>> in cluster 0  (0.0, 517)  (2.0, 513)  (1.0, 512)
>>> in cluster 1  (1.0, 251)  (2.0, 246)  (0.0, 244)

>> Age Category :

>>> in cluster 0  (23.0, 239)  (25.0, 161)  (24.0, 132)  (22.0, 99)  (26.0, 63)  (52.0, 60)  (28.0, 51)  (21.0, 48)  (58.0, 43)  (16.0, 42)  (29.0, 41)  (6.0, 40)  (54.0, 29)  (30.0, 29)  (60.0, 28)  (14.0, 28)  (18.0, 28)  (20.0, 27)  (56.0, 26)  (59.0, 26)  (46.0, 23)  (11.0, 22)  (50.0, 21)  (32.0, 21)  (34.0, 17)  (7.0, 17)  (38.0, 16)  (51.0, 15)  (82.0, 14)  (49.0, 14)  (47.0, 13)  (17.0, 13)  (57.0, 12)  (31.0, 12)  (10.0, 11)  (62.0, 11)  (40.0, 9)  (5.0, 9)  (44.0, 7)  (55.0, 7)  (12.0, 6)  (8.0, 6)  (19.0, 4)  (9.0, 2)
>>> in cluster 1  (25.0, 65)  (22.0, 50)  (24.0, 49)  (28.0, 40)  (21.0, 39)  (44.0, 38)  (34.0, 37)  (54.0, 30)  (26.0, 30)  (35.0, 29)  (52.0, 29)  (50

In [ ]:
clustering_information(data_f2 , 'f2',use_pca=False)

{ Clustring in Feature Set f2 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 605)  (0.0, 427)
>>> in cluster 1  (1.0, 840)  (0.0, 411)

>> Status Category :

>>> in cluster 0  (0.0, 356)  (1.0, 344)  (2.0, 332)
>>> in cluster 1  (2.0, 427)  (1.0, 419)  (0.0, 405)

>> Age Category :

>>> in cluster 0  (52.0, 81)  (28.0, 65)  (23.0, 62)  (25.0, 59)  (26.0, 52)  (54.0, 51)  (34.0, 39)  (50.0, 39)  (58.0, 37)  (44.0, 36)  (46.0, 36)  (22.0, 35)  (30.0, 29)  (60.0, 27)  (51.0, 25)  (29.0, 25)  (56.0, 25)  (40.0, 24)  (38.0, 24)  (55.0, 22)  (59.0, 22)  (57.0, 21)  (24.0, 20)  (35.0, 19)  (21.0, 18)  (31.0, 18)  (32.0, 16)  (82.0, 14)  (49.0, 14)  (47.0, 13)  (62.0, 11)  (20.0, 10)  (27.0, 10)  (33.0, 8)  (14.0, 4)  (19.0, 4)  (6.0, 4)  (10.0, 3)  (5.0, 3)  (7.0, 3)  (8.0, 2)  (11.0, 2)
>>> in cluster 1  (23.0, 199)  (25.0, 167)  (24.0, 161)  (22.0, 114)  (21.0, 69)  (6.0, 47)  (16.0, 44)  (26.0, 41)  (14.0, 37)  (20.0, 35)  (18.0, 28)  (7.0, 27)  (28.0, 26)  (11.

In [ ]:
clustering_information(data_f3 , 'f3',use_pca=True)

{ Clustring in Feature Set f3 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 1098)  (0.0, 69)
>>> in cluster 1  (0.0, 769)  (1.0, 347)

>> Status Category :

>>> in cluster 0  (2.0, 394)  (1.0, 392)  (0.0, 381)
>>> in cluster 1  (0.0, 380)  (1.0, 371)  (2.0, 365)

>> Age Category :

>>> in cluster 0  (23.0, 236)  (25.0, 157)  (24.0, 128)  (22.0, 89)  (26.0, 63)  (28.0, 45)  (21.0, 45)  (16.0, 42)  (29.0, 40)  (18.0, 28)  (30.0, 24)  (14.0, 23)  (32.0, 21)  (46.0, 19)  (20.0, 17)  (11.0, 16)  (34.0, 15)  (56.0, 14)  (17.0, 13)  (52.0, 12)  (54.0, 11)  (58.0, 10)  (6.0, 10)  (82.0, 9)  (5.0, 9)  (31.0, 9)  (40.0, 8)  (38.0, 8)  (59.0, 8)  (49.0, 7)  (10.0, 6)  (51.0, 5)  (62.0, 4)  (50.0, 4)  (8.0, 4)  (55.0, 2)  (57.0, 2)  (19.0, 2)  (60.0, 1)  (7.0, 1)
>>> in cluster 1  (52.0, 77)  (25.0, 69)  (22.0, 60)  (24.0, 53)  (54.0, 48)  (28.0, 46)  (44.0, 45)  (21.0, 42)  (6.0, 41)  (50.0, 41)  (34.0, 39)  (58.0, 33)  (26.0, 30)  (35.0, 29)  (7.0, 29)  (20.0, 28)  (

In [ ]:
clustering_information(data_f3 , 'f3',use_pca=False)

{ Clustring in Feature Set f3 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 355)  (0.0, 199)
>>> in cluster 1  (1.0, 1090)  (0.0, 639)

>> Status Category :

>>> in cluster 0  (1.0, 193)  (2.0, 181)  (0.0, 180)
>>> in cluster 1  (0.0, 581)  (2.0, 578)  (1.0, 570)

>> Age Category :

>>> in cluster 0  (52.0, 56)  (54.0, 44)  (58.0, 41)  (6.0, 31)  (60.0, 25)  (59.0, 24)  (50.0, 20)  (7.0, 20)  (20.0, 19)  (14.0, 18)  (46.0, 18)  (25.0, 17)  (23.0, 15)  (11.0, 15)  (24.0, 14)  (26.0, 13)  (56.0, 12)  (62.0, 11)  (82.0, 11)  (57.0, 11)  (47.0, 11)  (31.0, 11)  (38.0, 10)  (55.0, 8)  (30.0, 8)  (35.0, 8)  (44.0, 6)  (5.0, 6)  (34.0, 6)  (40.0, 5)  (28.0, 5)  (51.0, 4)  (22.0, 4)  (21.0, 4)  (9.0, 4)  (8.0, 4)  (33.0, 3)  (32.0, 3)  (29.0, 2)  (12.0, 2)  (10.0, 2)  (49.0, 2)  (16.0, 1)
>>> in cluster 1  (23.0, 246)  (25.0, 209)  (24.0, 167)  (22.0, 145)  (28.0, 86)  (21.0, 83)  (26.0, 80)  (34.0, 48)  (16.0, 43)  (29.0, 39)  (44.0, 39)  (30.0, 35)  (52.0, 33)  (

In [ ]:
clustering_information(data_f4 , 'f4',use_pca=True)

{ Clustring in Feature Set f4 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 546)  (0.0, 219)
>>> in cluster 1  (1.0, 1097)  (0.0, 683)

>> Status Category :

>>> in cluster 0  (0.0, 262)  (2.0, 252)  (1.0, 251)
>>> in cluster 1  (1.0, 600)  (2.0, 596)  (0.0, 584)

>> Age Category :

>>> in cluster 0  (23.0, 89)  (25.0, 56)  (24.0, 55)  (52.0, 44)  (26.0, 42)  (22.0, 39)  (28.0, 29)  (6.0, 24)  (14.0, 24)  (56.0, 21)  (21.0, 20)  (11.0, 20)  (54.0, 20)  (46.0, 19)  (58.0, 19)  (44.0, 17)  (57.0, 16)  (32.0, 16)  (30.0, 14)  (34.0, 13)  (40.0, 13)  (12.0, 12)  (35.0, 11)  (7.0, 10)  (31.0, 10)  (60.0, 9)  (33.0, 9)  (20.0, 9)  (29.0, 8)  (18.0, 8)  (17.0, 8)  (59.0, 8)  (16.0, 7)  (82.0, 6)  (50.0, 6)  (47.0, 5)  (55.0, 5)  (10.0, 4)  (49.0, 4)  (8.0, 4)  (62.0, 3)  (51.0, 3)  (5.0, 2)  (38.0, 2)  (27.0, 1)  (9.0, 1)
>>> in cluster 1  (23.0, 211)  (25.0, 184)  (24.0, 155)  (22.0, 113)  (28.0, 72)  (21.0, 70)  (26.0, 69)  (34.0, 47)  (52.0, 46)  (29.0, 40)  (5

In [ ]:
clustering_information(data_f4 , 'f4',use_pca=False)

{ Clustring in Feature Set f4 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 872)  (0.0, 473)
>>> in cluster 1  (1.0, 771)  (0.0, 429)

>> Status Category :

>>> in cluster 0  (0.0, 493)  (1.0, 433)  (2.0, 419)
>>> in cluster 1  (2.0, 429)  (1.0, 418)  (0.0, 353)

>> Age Category :

>>> in cluster 0  (23.0, 152)  (25.0, 133)  (22.0, 82)  (24.0, 80)  (26.0, 71)  (52.0, 67)  (54.0, 48)  (28.0, 39)  (21.0, 39)  (6.0, 37)  (56.0, 33)  (32.0, 27)  (30.0, 27)  (58.0, 26)  (7.0, 25)  (14.0, 25)  (57.0, 24)  (11.0, 24)  (60.0, 23)  (51.0, 22)  (46.0, 22)  (16.0, 21)  (55.0, 21)  (34.0, 21)  (40.0, 21)  (35.0, 20)  (44.0, 20)  (20.0, 17)  (29.0, 17)  (59.0, 17)  (31.0, 15)  (33.0, 15)  (18.0, 13)  (82.0, 12)  (12.0, 10)  (50.0, 10)  (38.0, 10)  (47.0, 9)  (8.0, 8)  (17.0, 8)  (5.0, 7)  (10.0, 6)  (62.0, 6)  (27.0, 5)  (19.0, 5)  (49.0, 4)  (9.0, 1)
>>> in cluster 1  (23.0, 148)  (24.0, 130)  (25.0, 107)  (22.0, 70)  (28.0, 62)  (21.0, 51)  (26.0, 40)  (34.0, 39)  (50

In [ ]:
clustering_information(data_f5 , 'f5',use_pca=True)

{ Clustring in Feature Set f5 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 603)  (0.0, 407)
>>> in cluster 1  (1.0, 512)  (0.0, 146)

>> Status Category :

>>> in cluster 0  (2.0, 342)  (0.0, 337)  (1.0, 331)
>>> in cluster 1  (1.0, 235)  (0.0, 215)  (2.0, 208)

>> Age Category :

>>> in cluster 0  (23.0, 112)  (22.0, 96)  (24.0, 95)  (25.0, 73)  (26.0, 61)  (21.0, 46)  (52.0, 45)  (16.0, 30)  (20.0, 30)  (59.0, 30)  (29.0, 29)  (28.0, 26)  (56.0, 25)  (32.0, 24)  (44.0, 23)  (11.0, 19)  (54.0, 17)  (7.0, 16)  (12.0, 15)  (35.0, 15)  (40.0, 15)  (46.0, 15)  (8.0, 15)  (82.0, 15)  (9.0, 15)  (19.0, 15)  (27.0, 15)  (5.0, 15)  (30.0, 15)  (55.0, 12)  (10.0, 9)  (33.0, 9)  (17.0, 7)  (51.0, 6)  (60.0, 2)  (6.0, 1)  (31.0, 1)  (47.0, 1)
>>> in cluster 1  (23.0, 90)  (24.0, 49)  (25.0, 47)  (28.0, 40)  (46.0, 30)  (31.0, 29)  (6.0, 28)  (54.0, 28)  (14.0, 27)  (22.0, 26)  (7.0, 25)  (51.0, 24)  (44.0, 22)  (26.0, 19)  (21.0, 19)  (11.0, 16)  (57.0, 15)  (34.0, 

In [ ]:
clustering_information(data_f5 , 'f5',use_pca=False)

{ Clustring in Feature Set f5 }

> number of clusters = 2

>> Gender Category :

>>> in cluster 0  (1.0, 554)  (0.0, 154)
>>> in cluster 1  (1.0, 561)  (0.0, 399)

>> Status Category :

>>> in cluster 0  (1.0, 251)  (0.0, 231)  (2.0, 226)
>>> in cluster 1  (2.0, 324)  (0.0, 321)  (1.0, 315)

>> Age Category :

>>> in cluster 0  (23.0, 93)  (25.0, 59)  (24.0, 54)  (28.0, 42)  (46.0, 30)  (6.0, 29)  (54.0, 29)  (31.0, 29)  (14.0, 27)  (22.0, 27)  (51.0, 25)  (7.0, 25)  (44.0, 25)  (26.0, 21)  (11.0, 20)  (21.0, 19)  (57.0, 15)  (34.0, 15)  (18.0, 15)  (47.0, 14)  (30.0, 13)  (60.0, 13)  (49.0, 13)  (56.0, 10)  (10.0, 9)  (32.0, 9)  (17.0, 9)  (29.0, 8)  (33.0, 7)  (55.0, 4)
>>> in cluster 1  (23.0, 109)  (22.0, 95)  (24.0, 90)  (25.0, 61)  (26.0, 59)  (21.0, 46)  (52.0, 45)  (16.0, 30)  (20.0, 30)  (59.0, 30)  (29.0, 28)  (28.0, 24)  (32.0, 21)  (56.0, 20)  (44.0, 20)  (7.0, 16)  (54.0, 16)  (12.0, 15)  (35.0, 15)  (40.0, 15)  (11.0, 15)  (46.0, 15)  (8.0, 15)  (82.0, 15)  (9.0, 15)  (19